In [16]:
import pandas as pd

In [18]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.svm import SVR
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report, confusion_matrix,accuracy_score

In [20]:
import numpy as np
from sklearn.multiclass import OneVsRestClassifier
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
import re
from sklearn.utils import shuffle
from sklearn.model_selection import StratifiedKFold, cross_val_score

<h3> Used Equation1 for the problem </h3>

In [23]:
input = pd.read_csv("../04_15/input_2500.csv", index_col=False)
columns_to_drop = ['Unnamed: 0']
input.drop(columns= columns_to_drop, inplace = True)

In [25]:
input.head()

,Sentiment_Score,Success,High_Increase,Positive_Performance,Positive_Sentiment,Positive_Emotions,Sold,Bullish,Strength,Moderate_Increase,...,Supply,Negative_Outcomes,Electricvehicle,Finance,EV_tech,tax,late,risk,problem,Sentiment_Calc
0,-1,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,...,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,24.9
1,-1,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19.0
2,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,-7.4
3,-1,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,-3.6
4,-1,1.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,...,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,8.4


In [27]:
#Sentiment_Score is actual labels

In [29]:
X = input.drop(columns = ["Sentiment_Score","Sentiment_Calc"])
y_original = input["Sentiment_Score"]

In [31]:
input['Eq_threshold'] = input['Sentiment_Calc'].apply(lambda x: 1 if x > 3 else -1 if x < -7 else 0)

In [33]:
y_positive_derived = input['Eq_threshold'].apply(lambda x: 100 if x == 1 else -100)
y_neutral_derived = input['Eq_threshold'].apply(lambda x: 100 if x == 0 else -100)
y_negative_derived = input['Eq_threshold'].apply(lambda x: 100 if x == -1 else -100)

In [35]:
X_train, X_test, y_train_orig, y_test_orig = train_test_split(X, y_original, test_size=0.2, random_state=28)

In [37]:
y_train_pos = y_positive_derived.iloc[X_train.index]
y_test_pos = y_positive_derived.iloc[X_test.index]

y_train_neu = y_neutral_derived.iloc[X_train.index]
y_test_neu = y_neutral_derived.iloc[X_test.index]

y_train_neg = y_negative_derived.iloc[X_train.index]
y_test_neg = y_negative_derived.iloc[X_test.index]

In [39]:
pos_svm = SVC(kernel='rbf', C=100, gamma=0.001, probability=True, class_weight='balanced', decision_function_shape='ovr')
neu_svm = SVC(kernel='rbf', C=1000000, gamma=0.00001, probability=True, class_weight='balanced', decision_function_shape='ovr')
neg_svm = SVC(kernel='rbf', C=1000000, gamma=0.0001, probability=True, class_weight='balanced', decision_function_shape='ovr')

In [41]:
pos_svm.fit(X_train, y_train_pos)

neu_svm.fit(X_train, y_train_neu)

neg_svm.fit(X_train, y_train_neg)

SVC(C=1000000, class_weight='balanced', gamma=0.0001, probability=True)

In [42]:
pos_scores = pos_svm.predict_proba(X_test)[:, 1]  # Probability of being 100
neu_scores = neu_svm.predict_proba(X_test)[:, 1]
neg_scores = neg_svm.predict_proba(X_test)[:, 1]

In [43]:
all_probs = np.vstack((pos_scores, neu_scores, neg_scores)).T
predicted_classes_idx = np.argmax(all_probs, axis=1)
index_to_sentiment = [1, 0, -1]  # 0 -> Positive, 1 -> Neutral, 2 -> Negative
predicted_classes = [index_to_sentiment[i] for i in predicted_classes_idx]

In [44]:
print("*"*10, " Test Results ", "*"*10)
print()
print(classification_report(y_test_orig, predicted_classes))

**********  Test Results  **********

              precision    recall  f1-score   support

          -1       1.00      0.01      0.02       122
           0       0.23      0.09      0.13       132
           1       0.51      0.93      0.66       246

    accuracy                           0.48       500
   macro avg       0.58      0.34      0.27       500
weighted avg       0.56      0.48      0.36       500



In [45]:
pos_scores_train = pos_svm.predict_proba(X_train)[:, 1]  # Probability of being 100
neu_scores_train = neu_svm.predict_proba(X_train)[:, 1]
neg_scores_train = neg_svm.predict_proba(X_train)[:, 1]

In [46]:
all_probs_train = np.vstack((pos_scores_train, neu_scores_train, neg_scores_train)).T
predicted_classes_train_idx = np.argmax(all_probs_train, axis=1)


index_to_sentiment = [1, 0, -1] 
predicted_classes_train = [index_to_sentiment[i] for i in predicted_classes_train_idx]

In [49]:
print("*"*10, " Train Results ", "*"*10)
print()
print(classification_report(y_train_orig, predicted_classes_train))

**********  Train Results  **********

              precision    recall  f1-score   support

          -1       0.71      0.01      0.02       500
           0       0.20      0.09      0.13       548
           1       0.51      0.93      0.66       952

    accuracy                           0.47      2000
   macro avg       0.48      0.34      0.27      2000
weighted avg       0.48      0.47      0.35      2000



<h3> Solving it as a Discrete Problem </h3>

In [56]:
def generate_binary_labels(y, target_class):
    return np.where(y == target_class, 100, -100)

<h3> Positive </h3>

In [59]:
svm_pos = SVC(kernel='rbf', C=100, gamma=0.001, probability=True, class_weight='balanced', decision_function_shape='ovr')

In [61]:
X_train, X_test, y_train_orig, y_test_orig = train_test_split(X, y_original, test_size=0.2, random_state=28)
y_positive = y_original.apply(lambda x : 100 if x == 1 else -100)

In [63]:
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

train_precisions = []
train_recalls = []
train_f1s = []
test_precisions = []
test_recalls = []
test_f1s = []

In [65]:
for fold, (train_index, test_index) in enumerate(kf.split(X, y_positive)):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y_positive.iloc[train_index], y_positive.iloc[test_index]
    svm_pos.fit(X_train, y_train)
    y_train_pred = svm_pos.predict(X_train)
    y_test_pred = svm_pos.predict(X_test)

    train_precision = precision_score(y_train, y_train_pred, average='weighted')
    train_recall = recall_score(y_train, y_train_pred, average='weighted')
    train_f1 = f1_score(y_train, y_train_pred, average='weighted')
    
    test_precision = precision_score(y_test, y_test_pred, average='weighted')
    test_recall = recall_score(y_test, y_test_pred, average='weighted')
    test_f1 = f1_score(y_test, y_test_pred, average='weighted')
    
    train_precisions.append(train_precision)
    train_recalls.append(train_recall)
    train_f1s.append(train_f1)
    test_precisions.append(test_precision)
    test_recalls.append(test_recall)
    test_f1s.append(test_f1)
    
    print(f'Fold {fold + 1}:')
    print(f'  Train Precision: {train_precision:.3f}')
    print(f'  Train Recall: {train_recall:.3f}')
    print(f'  Train F1 Score: {train_f1:.3f}')
    print(f'  Test Precision: {test_precision:.3f}')
    print(f'  Test Recall: {test_recall:.3f}')
    print(f'  Test F1 Score: {test_f1:.3f}')

Fold 1:
  Train Precision: 0.664
  Train Recall: 0.664
  Train F1 Score: 0.664
  Test Precision: 0.638
  Test Recall: 0.638
  Test F1 Score: 0.637
Fold 2:
  Train Precision: 0.654
  Train Recall: 0.654
  Train F1 Score: 0.654
  Test Precision: 0.648
  Test Recall: 0.648
  Test F1 Score: 0.648
Fold 3:
  Train Precision: 0.665
  Train Recall: 0.665
  Train F1 Score: 0.665
  Test Precision: 0.636
  Test Recall: 0.636
  Test F1 Score: 0.636
Fold 4:
  Train Precision: 0.665
  Train Recall: 0.665
  Train F1 Score: 0.665
  Test Precision: 0.634
  Test Recall: 0.634
  Test F1 Score: 0.633
Fold 5:
  Train Precision: 0.660
  Train Recall: 0.659
  Train F1 Score: 0.660
  Test Precision: 0.660
  Test Recall: 0.660
  Test F1 Score: 0.660


In [66]:
print('\nAverage Results:')
print(f'Train Precision: {np.mean(train_precisions):.3f} ± {np.std(train_precisions):.3f}')
print(f'Train Recall: {np.mean(train_recalls):.3f} ± {np.std(train_recalls):.3f}')
print(f'Train F1 Score: {np.mean(train_f1s):.3f} ± {np.std(train_f1s):.3f}')
print(f'Test Precision: {np.mean(test_precisions):.3f} ± {np.std(test_precisions):.3f}')
print(f'Test Recall: {np.mean(test_recalls):.3f} ± {np.std(test_recalls):.3f}')
print(f'Test F1 Score: {np.mean(test_f1s):.3f} ± {np.std(test_f1s):.3f}')


Average Results:
Train Precision: 0.662 ± 0.004
Train Recall: 0.662 ± 0.004
Train F1 Score: 0.662 ± 0.004
Test Precision: 0.643 ± 0.010
Test Recall: 0.643 ± 0.010
Test F1 Score: 0.643 ± 0.010


<h3> Neutral</h3>

In [81]:
svm_neu = SVC(kernel='rbf', C=1000000, gamma=0.00001, probability=True, class_weight='balanced', decision_function_shape='ovr')

In [83]:
X_train, X_test, y_train_orig, y_test_orig = train_test_split(X, y_original, test_size=0.2, random_state=28)
y_neutral = y_original.apply(lambda x : 100 if x == 0 else -100)

In [85]:
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

train_precisions = []
train_recalls = []
train_f1s = []
test_precisions = []
test_recalls = []
test_f1s = []

In [87]:
for fold, (train_index, test_index) in enumerate(kf.split(X, y_neutral)):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y_neutral.iloc[train_index], y_neutral.iloc[test_index]
    svm_neu.fit(X_train, y_train)
    y_train_pred = svm_neu.predict(X_train)
    y_test_pred = svm_neu.predict(X_test)

    train_precision = precision_score(y_train, y_train_pred, average='weighted')
    train_recall = recall_score(y_train, y_train_pred, average='weighted')
    train_f1 = f1_score(y_train, y_train_pred, average='weighted')
    
    test_precision = precision_score(y_test, y_test_pred, average='weighted')
    test_recall = recall_score(y_test, y_test_pred, average='weighted')
    test_f1 = f1_score(y_test, y_test_pred, average='weighted')
    
    train_precisions.append(train_precision)
    train_recalls.append(train_recall)
    train_f1s.append(train_f1)
    test_precisions.append(test_precision)
    test_recalls.append(test_recall)
    test_f1s.append(test_f1)
    
    print(f'Fold {fold + 1}:')
    print(f'  Train Precision: {train_precision:.3f}')
    print(f'  Train Recall: {train_recall:.3f}')
    print(f'  Train F1 Score: {train_f1:.3f}')
    print(f'  Test Precision: {test_precision:.3f}')
    print(f'  Test Recall: {test_recall:.3f}')
    print(f'  Test F1 Score: {test_f1:.3f}')

Fold 1:
  Train Precision: 0.691
  Train Recall: 0.550
  Train F1 Score: 0.573
  Test Precision: 0.666
  Test Recall: 0.552
  Test F1 Score: 0.576
Fold 2:
  Train Precision: 0.672
  Train Recall: 0.590
  Train F1 Score: 0.612
  Test Precision: 0.654
  Test Recall: 0.564
  Test F1 Score: 0.588
Fold 3:
  Train Precision: 0.663
  Train Recall: 0.536
  Train F1 Score: 0.560
  Test Precision: 0.667
  Test Recall: 0.532
  Test F1 Score: 0.555
Fold 4:
  Train Precision: 0.680
  Train Recall: 0.551
  Train F1 Score: 0.574
  Test Precision: 0.640
  Test Recall: 0.504
  Test F1 Score: 0.528
Fold 5:
  Train Precision: 0.679
  Train Recall: 0.568
  Train F1 Score: 0.591
  Test Precision: 0.630
  Test Recall: 0.536
  Test F1 Score: 0.562


In [89]:
print('\nAverage Results:')
print(f'Train Precision: {np.mean(train_precisions):.3f} ± {np.std(train_precisions):.3f}')
print(f'Train Recall: {np.mean(train_recalls):.3f} ± {np.std(train_recalls):.3f}')
print(f'Train F1 Score: {np.mean(train_f1s):.3f} ± {np.std(train_f1s):.3f}')
print(f'Test Precision: {np.mean(test_precisions):.3f} ± {np.std(test_precisions):.3f}')
print(f'Test Recall: {np.mean(test_recalls):.3f} ± {np.std(test_recalls):.3f}')
print(f'Test F1 Score: {np.mean(test_f1s):.3f} ± {np.std(test_f1s):.3f}')


Average Results:
Train Precision: 0.677 ± 0.009
Train Recall: 0.559 ± 0.018
Train F1 Score: 0.582 ± 0.018
Test Precision: 0.652 ± 0.015
Test Recall: 0.538 ± 0.020
Test F1 Score: 0.562 ± 0.020


<h3> Negative</h3>

In [91]:
svm_neg = SVC(kernel='rbf', C=1000000, gamma=0.0001, probability=True, class_weight='balanced', decision_function_shape='ovr')

In [93]:
X_train, X_test, y_train_orig, y_test_orig = train_test_split(X, y_original, test_size=0.2, random_state=28)
y_negative = y_original.apply(lambda x : 100 if x == -1 else -100)

In [95]:
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

train_precisions = []
train_recalls = []
train_f1s = []
test_precisions = []
test_recalls = []
test_f1s = []

In [97]:
for fold, (train_index, test_index) in enumerate(kf.split(X, y_negative)):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y_negative.iloc[train_index], y_negative.iloc[test_index]
    svm_neg.fit(X_train, y_train)
    y_train_pred = svm_neg.predict(X_train)
    y_test_pred = svm_neg.predict(X_test)

    train_precision = precision_score(y_train, y_train_pred, average='weighted')
    train_recall = recall_score(y_train, y_train_pred, average='weighted')
    train_f1 = f1_score(y_train, y_train_pred, average='weighted')
    
    test_precision = precision_score(y_test, y_test_pred, average='weighted')
    test_recall = recall_score(y_test, y_test_pred, average='weighted')
    test_f1 = f1_score(y_test, y_test_pred, average='weighted')
    
    train_precisions.append(train_precision)
    train_recalls.append(train_recall)
    train_f1s.append(train_f1)
    test_precisions.append(test_precision)
    test_recalls.append(test_recall)
    test_f1s.append(test_f1)
    
    print(f'Fold {fold + 1}:')
    print(f'  Train Precision: {train_precision:.3f}')
    print(f'  Train Recall: {train_recall:.3f}')
    print(f'  Train F1 Score: {train_f1:.3f}')
    print(f'  Test Precision: {test_precision:.3f}')
    print(f'  Test Recall: {test_recall:.3f}')
    print(f'  Test F1 Score: {test_f1:.3f}')

Fold 1:
  Train Precision: 0.820
  Train Recall: 0.775
  Train F1 Score: 0.787
  Test Precision: 0.781
  Test Recall: 0.740
  Test F1 Score: 0.753
Fold 2:
  Train Precision: 0.806
  Train Recall: 0.762
  Train F1 Score: 0.775
  Test Precision: 0.747
  Test Recall: 0.704
  Test F1 Score: 0.718
Fold 3:
  Train Precision: 0.812
  Train Recall: 0.767
  Train F1 Score: 0.780
  Test Precision: 0.775
  Test Recall: 0.708
  Test F1 Score: 0.726
Fold 4:
  Train Precision: 0.802
  Train Recall: 0.757
  Train F1 Score: 0.770
  Test Precision: 0.787
  Test Recall: 0.740
  Test F1 Score: 0.754
Fold 5:
  Train Precision: 0.806
  Train Recall: 0.761
  Train F1 Score: 0.774
  Test Precision: 0.755
  Test Recall: 0.704
  Test F1 Score: 0.720


In [98]:
print('\nAverage Results:')
print(f'Train Precision: {np.mean(train_precisions):.3f} ± {np.std(train_precisions):.3f}')
print(f'Train Recall: {np.mean(train_recalls):.3f} ± {np.std(train_recalls):.3f}')
print(f'Train F1 Score: {np.mean(train_f1s):.3f} ± {np.std(train_f1s):.3f}')
print(f'Test Precision: {np.mean(test_precisions):.3f} ± {np.std(test_precisions):.3f}')
print(f'Test Recall: {np.mean(test_recalls):.3f} ± {np.std(test_recalls):.3f}')
print(f'Test F1 Score: {np.mean(test_f1s):.3f} ± {np.std(test_f1s):.3f}')


Average Results:
Train Precision: 0.809 ± 0.006
Train Recall: 0.765 ± 0.006
Train F1 Score: 0.777 ± 0.006
Test Precision: 0.769 ± 0.015
Test Recall: 0.719 ± 0.017
Test F1 Score: 0.734 ± 0.016


<h3> Evaluation </h3>

In [108]:
pos_scores = svm_pos.predict_proba(X_test)[:, 1]  # Probability of being 100
neu_scores = svm_neu.predict_proba(X_test)[:, 1]
neg_scores = svm_neg.predict_proba(X_test)[:, 1]

In [110]:
all_probs = np.vstack((pos_scores, neu_scores, neg_scores)).T
predicted_classes_train_idx = np.argmax(all_probs, axis=1)


index_to_sentiment = [1, 0, -1] 
predicted_classes_train = [index_to_sentiment[i] for i in predicted_classes_train_idx]

In [114]:
print("*"*10, " Test Results ", "*"*10)
print()
print(classification_report(y_test_orig, predicted_classes_train))

**********  Test Results  **********

              precision    recall  f1-score   support

          -1       0.21      0.16      0.18       122
           0       0.25      0.08      0.12       132
           1       0.50      0.74      0.60       246

    accuracy                           0.43       500
   macro avg       0.32      0.33      0.30       500
weighted avg       0.36      0.43      0.37       500



In [119]:
pos_scores = svm_pos.predict_proba(X_train)[:, 1]  # Probability of being 100
neu_scores = svm_neu.predict_proba(X_train)[:, 1]
neg_scores = svm_neg.predict_proba(X_train)[:, 1]

In [121]:
all_probs = np.vstack((pos_scores, neu_scores, neg_scores)).T
predicted_classes_train_idx = np.argmax(all_probs, axis=1)


index_to_sentiment = [1, 0, -1] 
predicted_classes_train = [index_to_sentiment[i] for i in predicted_classes_train_idx]

In [123]:
print("*"*10, " Test Results ", "*"*10)
print()
print(classification_report(y_train_orig, predicted_classes_train))

**********  Test Results  **********

              precision    recall  f1-score   support

          -1       0.24      0.22      0.23       500
           0       0.25      0.06      0.09       548
           1       0.48      0.72      0.57       952

    accuracy                           0.41      2000
   macro avg       0.33      0.33      0.30      2000
weighted avg       0.36      0.41      0.36      2000

